# Rješavanje problema trgovačkog putnika koristeći genetički algoritam

### Nešt o trgovačkom putniku

Korišteni moduli

In [2]:
import numpy as np
import random
import math
import networkx as nx #možda staviti poslije
import pandas as pd #možda staviti na kraj
import matplotlib.pyplot as plt #možda poslije

Klasa Map koja predstavlja graf s gradovima

In [ ]:
class Map():
    def __init__(self, n):
        self.num_of_cities = n
        self.xy = np.empty((n,2))
        for i in range(n):
            self.xy[i] = random.randint(0, 200), random.randint(0,200)
        self.distances = self.calculate_distances()
        

    def calculate_distances():
        distances = np.empty((self.num_of_cities, self.num_of_cities))
        for i in range(self.num_of_cities):
            p = self.xy[i]
            for j in range(self.num_of_cities):
                q = self.xy[j]
                distances[i][j] = math.dist(p,q)
        return distances
    
    def draw_graph():
        fig, (ax1, ax2) = plt.subplot()
        ax1.set_title('Cities')
        ax1.plt.scatter(self.xy[:,0], self.xy[:, 1])
        #ax2 je nx graf
        fig
        

Inicijalizacija mape

In [ ]:
map = Map(50)
map.draw_graph()

### Gen algoritam
Stavljam kod kak objašnjavam algoritam

Klasa ruta

In [ ]:
class Route():
    def __init__(self, n, graph):
        self.num_of_cities = n
        self.fitness = 0
        self.distance = 0
        self.map = graph
        self.path = np.random.permutation(self.num_of_cities)

    def calcDistance(self):
        for i in range(self.num_of_cities-1):
            self.distance += self.map.distances[self.path[i]][self.path[i+1]]
        self.distance += self.map.distances[self.path[0]][self.path[self.num_of_cities-1]]

    def calcFitness(self):
        self.calcDistance()
        self.fitness = 1/self.distance * 1000

    def clone(self):
        new_route = Route(self.num_of_cities, self.map)
        new_route.path = self.path
        return new_route
        

#### Populacija i jedinke

In [ ]:
class  Population():
    def __init__(self, n, m, map):
        self.map = map
        self.num_of_routes = n
        self.num_of_cities = m
        self.routes = []
        for i in range(n):
            self.routes.append(Route(m,map))
        self.fitness = 0
        self.best_distance_in_generation = []
        self.best_route = ""
        self.num_of_generation = 1


    def find_best_route(self):
        best_route = self.routes[0]
        for route in self.routes:
            if best_route.fitness < route.fitness:
                best_route = route
        self.best_distance_in_generation.append(best_route.distance)
        self.best_route = best_route.clone()
        self.best_route.distance = best_route.distance

Fitness funkcija

In [ ]:
def calcFitness(population):
    fitness = 0
    for route in population.routes:
        route.calcFitness()
        fitness += route.fitness
    return fitness

#### Selection

roulette wheel, slika (od zmije)

In [ ]:
def select_parent(population):
    rand = random.randint(0, population.fitness)
        sum = 0
        for route in population.routes
            sum += route.fitness
            if sum >= rand:
                return route

#### Crossover

napraviti sliku

In [ ]:
def crossover(parent1, parent2):
    child = Route(num_of_cities, self.map)
    gene1 = random.randint(0, num_of_cities)
    gene2 = random.randint(0, num_of_cities)

    start = min(gene1, gene2)
    end = max(gene1, gene2) + 1

    child.path[start:end] = parent1.path[start:end]
    child.path[:start] = child.path[end:] = -1

    for gene in parent2.path:
        if gene not in child.path:
            for i in range(self.num_of_cities):
                if child.path[i] == -1:
                    child.path[i] = gene
                    break
        return child

#### Mutacija

stopa mutacije bla bla

In [ ]:
def mutate(route, mutation_ratre): 
        for i in range(route.num_of_cities):
            if random.random() < mutation_rate:
                j = random.randrange(0, route.num_of_cities)
                route.path[i], route.path[j] = route.path[j], route.path[i]

#### Cijeli gen algoritam

In [ ]:
def geneticAlgorithm(population, mutation_rate):
    population.find_best_route()
    new_routes = [population.best_route.clone()]
    for i in range(self.num_of_routes):
        parent1 = select_parent(population)
        parent2 = select_parent(population)
        child = crossover(parent1, parent2)
        child = mutate(child, mutation rate)
        new_routes.append(child)
    return new_routes

Main funkcija

In [ ]:
def travel(num_of_routes, mutation_rate, num_of_generations, map):
    population = Population(num_of_routes, map.num_of_cities, map)
    #map.draw_coordinates()
    for i in range(num_of_generations):
        population.fitness = calcFitness(population)
        population.routes = geneticAlgorithm(population, mutation_rate)
         #možda iscrtaj svaku iteraciju (svaku 50 ili 10 ili ...)???
    population.draw_distances_in_generation() 
    #iscrtaj put na grafu, ispiši duljinu
    

Pokretanje

In [ ]:
travel(100, 0.1, 500, map) #dodati widgetse

Graf kretanja udaljenosti

In [ ]:
def draw_distance_over_iterations(population):
    fig, ax = plt.subplots()
    ax.set_title('Distance over generations')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Distance')
    x = np.arange(1, population.num_of_generations+1)
    ax.plot(x, population.best_distance_in_generation)
    fig

Moji spremljeni i učitani podaci, s npr 5000 generacija (iscrtaj put i graf udaljenosti), ili random ili stvarni podaci ili oboje

Primjer sa stvarnim gradovima i koordinatama (odaberi random gradove, iscrtaj kartu s imenima, graf udaljenosti)

# Bibliografija
1. https://www.theprojectspot.com/tutorial-post/applying-a-genetic-algorithm-to-the-travelling-salesman-problem/5
2. https://simplemaps.com/data/world-cities
3. ono kaj sam imala za zmiju